In [1]:
import ipywidgets as widgets
from IPython.display import display

import glob2
import pandas as pd

In [2]:
pd.set_option('display.max_rows', 100)
pd.set_option('display.max_columns', 50)
pd.set_option('display.max_colwidth', 100) #50) # default

In [3]:
def update_make_model(make_model):
    global filepath
    filepath = './output/*{}*.csv'.format(make_model)
    print(filepath)
    return None

make_models = ['Mazda-Miata', 'BMW-Z4', 'Nissan-370Z', 'Toyota-4Runner']
drop_down = \
widgets.Dropdown(
    options = make_models,
    value = make_models[0],
    description = 'Select Make-Model',
    disabled=False,
)

#display(drop_down)
widgets.interactive(update_make_model, make_model=drop_down)

interactive(children=(Dropdown(description='Select Make-Model', options=('Mazda-Miata', 'BMW-Z4', 'Nissan-370Z…

In [83]:
filenames = glob2.glob(filepath)
#print(filenames)

In [84]:
df = pd.read_csv(filenames[0])
for filename in filenames[1:]:
    df = pd.concat([df, pd.read_csv(filename)], ignore_index=True, sort=False)

In [85]:
df.index.name = 'orig_index'

In [86]:
#df.head(2)

In [87]:
agg = df.groupby('vin').agg({'date': ['max', 'min', 'count']})
agg.columns = ['max_date', 'min_date', 'count_date']

# add aggregations to row by vin
df_stats = df.join(agg, on='vin')

In [88]:
#df_stats.head(2)

In [89]:
def avail(row, date_max):
    if row['count_date'] == 1 and row['date'] == date_max:
        return 'new listing'
    if row['max_date'] < date_max:
        return 'listing not avail'
    return 'listing avail'

date_max = df['date'].max()
date_max

df_stats['availability'] = df_stats.apply(avail, axis=1, date_max=date_max)


In [90]:
#df_stats.head(2)

In [91]:
# order rows by vin and date to setup for calculating the price difference between dates
df_stats = df_stats.sort_values(by=['vin', 'date'])


In [92]:
#df_stats.head(15)

In [93]:
df_stats_price_diff = df_stats.groupby('vin')['vehicle_price'].rolling(2).apply(lambda price: price[1] - price[0], raw=True).to_frame('price_diff').reset_index()


In [94]:
#df_stats_price_diff.head(5)

In [95]:
assert len(df_stats) == len(df_stats_price_diff)

In [96]:
# join(..., left_on='index', right_on='level_1') is sufficient
# adding 'vin' is redundant, but keeps from adding duplicate columns vin_x, vin_y to dataframe
df_stats = df_stats.merge(df_stats_price_diff, left_on=['vin', 'orig_index'], right_on=['vin', 'orig_index'])


In [97]:
#df_stats.head(5)

In [98]:
#df_stats.columns

In [99]:
#['year_make_model_1', ''index', detail_url', 'vin', 'price_rating', 'vehicle_price', 'year_make_model_2', 'trim', 'location', 'mileage', 'exterior_color',
# 'interior_color', 'mpg', 'engine', 'transmission', 'drive_type', 'fuel_type', 'accident_check', 'usage', 'title', 'number_of_owners', 'date', 'max_date', 'min_date', 
# 'count_date', 'availability', 'price_diff']

# columns wanted for analysis
df_less_columns = df_stats[['year_make_model_1', 'vin', 'price_rating', 'vehicle_price', 'price_diff', 'trim', 'location', 'mileage', 
                 'exterior_color', 'interior_color', 'transmission', 'accident_check', 'usage', 'title', 'number_of_owners', 'date', 'availability', 'detail_url']]

In [100]:
#df_less_columns.head(5)

In [101]:
# various row filters
filter1 = df_less_columns['accident_check'] == 0 # no accidents
filter2 = df_less_columns['number_of_owners'] < 3 # less than three owners
filter3 = df_less_columns['title'] == 'Clean' # only interested in clean titles
filter4 = df_less_columns['availability'] != 'listing not avail' # not interested in listings that are no longer available


In [102]:
# note, for very large datasets it would be better to save new dataframe with the applied filter(s) instead of repeated filtering

#df_less_columns[filter1
#df_less_columns[filter1 & filter2]
#df_less_columns[filter1 & filter2 & filter3]
vins = list(set(df_less_columns[filter1 & filter2 & filter3 & filter4]['vin']))
list(enumerate(vins))

[(0, 'JTEBU5JR2C5108050'),
 (1, 'JTEBU5JR4B5064969'),
 (2, 'JTEBU5JR7A5010385'),
 (3, 'JTEBU5JR1C5109657'),
 (4, 'JTEBU5JR9E5195657'),
 (5, 'JTEBU5JR2A5010715'),
 (6, 'JTEBU5JR1E5186516'),
 (7, 'JTEBU5JR2D5131684'),
 (8, 'JTEBU5JR9F5213933'),
 (9, 'JTEBU5JR1D5118747'),
 (10, 'JTEBU5JR7D5129218'),
 (11, 'JTEBU5JR0A5015377'),
 (12, 'JTEBU5JR4B5071033')]

In [103]:
def update_vin(vin):
    #print('create dataframe for {} csv files'.format(make_model))
    global sel_vin
    sel_vin = vin
    #print(sel_vin)
    return None

vins = list(set(df_less_columns[filter1 & filter2 & filter3 & filter4]['vin']))
drop_down = \
widgets.Dropdown(
    options = vins,
    value = vins[0],
    description = 'Select vins',
    disabled=False,
)

#display(drop_down)
widgets.interactive(update_vin, vin=drop_down)

interactive(children=(Dropdown(description='Select vins', options=('JTEBU5JR2C5108050', 'JTEBU5JR4B5064969', '…

In [104]:
filter5 = df_less_columns['vin'] == sel_vin
df_less_columns[filter5] #['detail_url']

,year_make_model_1,vin,price_rating,vehicle_price,price_diff,trim,location,mileage,exterior_color,interior_color,transmission,accident_check,usage,title,number_of_owners,date,availability,detail_url
191,2012 Toyota 4Runner,JTEBU5JR2C5108050,Great Price,18250,NaN,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-13 08:25:29.186547,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
192,2012 Toyota 4Runner,JTEBU5JR2C5108050,Great Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-14 06:56:26.527273,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
193,2012 Toyota 4Runner,JTEBU5JR2C5108050,Great Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-16 07:36:38.804909,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
194,2012 Toyota 4Runner,JTEBU5JR2C5108050,Great Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-16 20:15:13.448345,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
195,2012 Toyota 4Runner,JTEBU5JR2C5108050,Great Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-17 09:57:02.609133,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
196,2012 Toyota 4Runner,JTEBU5JR2C5108050,Great Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-18 10:00:03.205843,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
197,2012 Toyota 4Runner,JTEBU5JR2C5108050,Great Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-21 09:59:35.636160,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
198,2012 Toyota 4Runner,JTEBU5JR2C5108050,Excellent Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-23 18:45:13.538394,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
199,2012 Toyota 4Runner,JTEBU5JR2C5108050,Excellent Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-25 08:18:41.949178,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/
200,2012 Toyota 4Runner,JTEBU5JR2C5108050,Great Price,18250,0.0,SR5 V6 4WD,"Seekonk, MA",91294,Classic Silver Metallic,Black/Graphite,Automatic,0,Personal or Rental Use,Clean,2,2020-03-29 20:03:24.703305,listing avail,https://penfed.truecar.com/used-cars-for-sale/listing/JTEBU5JR2C5108050/2012-toyota-4runner/


In [105]:
# filter by index range
#df_less_columns[filter1 & filter2 & filter3 & filter4].loc[66:79] # view range of listings of interest

In [106]:
# filter by index row
#df_less_columns[filter1 & filter2 & filter3 & filter4].loc[109] # view listings details for a specific listing